# NFT Authenticity and Prediction using Sentiment Analysis and Deep Learning

## Imports

In [88]:
import json
import asyncio
import tqdm
import tqdm.asyncio
import nest_asyncio
import pandas as pd
nest_asyncio.apply()
import os
from dotenv import load_dotenv
load_dotenv()
from pprint import pprint
from datetime import datetime
from dateutil.relativedelta import relativedelta
from aiohttp import ClientSession, TCPConnector

## Data Collection

Sample Data

In [148]:
sample_slugs = ['evolved-apes-inc','oraclenft','boredapeyachtclub','neo-tokyo-identities','cool-cats-nft','cryptopunks','veefriends']
slug_date = {}


### OpenSea Metrics from Collection Slugs

Asynchronous Data Collection

In [164]:
async def fetchData(url,session):
    async with session.get(url) as response:
        return await response.read()

async def transactionScrape():
    tasks = []
    responses = []
    connector = TCPConnector(limit_per_host=2)
    url = "https://api.opensea.io/api/v1/events?collection_slug={slug}&only_opensea=false&event_type=successful&limit=300&occurred_after={start}&occurred_before={end}"
    headers={"Accept": "application/json", "X-API-KEY": os.getenv('OPENSEA_API_KEY')}
    async with ClientSession(connector=connector, headers=headers) as session:
        for i in range(len(sample_slugs)):
            start_date = datetime.fromisoformat(slug_date[sample_slugs[i]].split('T')[0])
            end_date = start_date + relativedelta(months=1)
            task = asyncio.ensure_future(fetchData(url.format(slug=sample_slugs[i],start=start_date.timestamp(), end=end_date.timestamp()),session))
            tasks.append(task)
        for f in tqdm.tqdm(asyncio.as_completed(tasks), total=len(tasks)):
            responses.append(await f)
    for response in responses:
        response = json.loads(response.decode('utf8'))
        try:
            pprint(response['asset_events'][0]['collection_slug'])
            total_price = 0
            for event in response['asset_events']:
                total_price += float(event['total_price'])/1000000000000000000
            average_price = total_price/len(response['asset_events'])
            pprint(f'Average 1 month after mint price: {average_price}')  #OPENSEA API LIMITS EVENTS TO 300 SO REALLY ITS THE LAST 300 TRANSACTIONS IN THE MONTH
        except:
            print(response)

async def collectionScrape():
    tasks = []
    responses = []
    connector = TCPConnector()
    url = "https://api.opensea.io/api/v1/collection/{}"
    async with ClientSession(connector=connector) as session:
        for i in range(len(sample_slugs)):
            task = asyncio.ensure_future(fetchData(url.format(sample_slugs[i]),session))
            tasks.append(task)
        for f in tqdm.tqdm(asyncio.as_completed(tasks), total=len(tasks)):
            responses.append(await f)
    for response in responses:
        response = json.loads(response.decode('utf8'))
        try:
            name = response['collection']['name']
            floor = response['collection']['stats']['floor_price']
            volume = response['collection']['stats']['total_volume']
            date_created = response['collection']['created_date']
            slug_date[response['collection']['slug']] = date_created
            print(f'{name}: Floor: {floor}ETH --- Total Volume: {volume}ETH --- Created {date_created}')
        except:
            print(response)


Run Collection Script

In [149]:
loop = asyncio.get_event_loop()
future = asyncio.ensure_future(collectionScrape())
loop.run_until_complete(future)

100%|██████████| 7/7 [00:00<00:00, 26.03it/s]

CryptoPunks: Floor: NoneETH --- Total Volume: 730031.9212492915ETH --- Created 2019-04-26T22:13:09.691572
Neo Tokyo Identities: Floor: 16.0ETH --- Total Volume: 11015.147364958138ETH --- Created 2021-10-04T15:14:32.376341
Cool Cats NFT: Floor: 6.99ETH --- Total Volume: 58695.75946848873ETH --- Created 2021-06-27T09:03:35.403074
VeeFriends: Floor: 8.39ETH --- Total Volume: 31968.70620255153ETH --- Created 2021-05-11T18:22:19.398578
Bored Ape Yacht Club: Floor: 50.69ETH --- Total Volume: 246279.89621350318ETH --- Created 2021-04-22T23:14:03.967121
Oracle (official): Floor: 0.0ETH --- Total Volume: 27.59212000000001ETH --- Created 2021-09-22T10:53:26.948567
Evolved Apes Inc: Floor: 0.0009ETH --- Total Volume: 1155.88726411926ETH --- Created 2021-09-23T12:30:03.410309


In [166]:
loop = asyncio.get_event_loop()
future = asyncio.ensure_future(transactionScrape())
loop.run_until_complete(future)

100%|██████████| 7/7 [00:03<00:00,  1.75it/s]


'evolved-apes-inc'
'Average 1 month after mint price: 0.018939976401812286'
'oraclenft'
'Average 1 month after mint price: 0.04743173333333336'
'boredapeyachtclub'
'Average 1 month after mint price: 0.5512016432206388'
'cool-cats-nft'
'Average 1 month after mint price: 0.7231994174796048'
'neo-tokyo-identities'
'Average 1 month after mint price: 11.5224924328882'
'veefriends'
'Average 1 month after mint price: 5.033078654164987'
'cryptopunks'
'Average 1 month after mint price: 0.5936562130177515'
